In [1]:
import os
import mido
import utils ##
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

Datos preprocesados:
https://drive.google.com/drive/folders/1jzfK-VM_Qb76K31Y7iMpsi1JeQ7QqU5W?usp=sharing

# Importar Datos

In [2]:
df_midis_info = pd.read_csv('datos_procesados/midis/midis_info.csv', index_col=0)
df_midis_notas = pd.read_csv('datos_procesados/midis/midis_notas.csv', index_col=0)

In [3]:
# Tipos de MIDI
df_non_standard = pd.read_csv('datos_procesados/midis/tipos/no_estandar.csv', index_col=0)
df_standard = pd.read_csv('datos_procesados/midis/tipos/estandar.csv', index_col=0)

In [4]:
# Usar los audios con expresión MIDI estándar
df_midi = df_standard.merge(df_midis_info, how='left', left_index=True, right_index=True)
df_midi = df_midi.drop(columns=['problem?'])
df_midi = df_midi.rename({'velocity':'velocity_start'}, axis=1)

df_notas = df_midis_notas.loc[df_midi.index]

# MIDIS

In [5]:
lowest_time = df_midi.min_time.min()
print(lowest_time, 's')

0.125 s


### Filtrar frecuencias
Sólo se conservan aquellas que son alcanzables por una voz humana

In [6]:
### Filtra notas que no son ejecutadas
notas_mask = (df_notas.sum() > 0)
notas_ejecutadas = notas_mask[notas_mask]

# Transforma en DataFrame
notas_ejecutadas = notas_ejecutadas.index.str.extract('(^\d+) \((\w#?\d)\)') # Separa la clave MIDI del Cifrado
notas_ejecutadas = notas_ejecutadas.rename({0:'midi-ET', 1: 'cifrado'}, axis=1) # Renombrar
notas_ejecutadas['midi-ET'] = notas_ejecutadas['midi-ET'].astype(int) # Regresar a numérico
notas_ejecutadas['herts-ET'] = utils.preprocess.nota_a_frequencia(notas_ejecutadas['midi-ET']) # Convertir a frecuencia

In [7]:
# Calcular cotas de filtrado
hz_std = notas_ejecutadas['herts-ET'].std() 

hz_lb = np.floor( notas_ejecutadas['herts-ET'].min() * (1-0.5) )
hz_ub = np.ceil( notas_ejecutadas['herts-ET'].max() + 0.5*hz_std )

print(f'Rango de frecuencias utilizable (Hz): ({hz_lb}, {hz_ub})')

Rango de frecuencias utilizable (Hz): (18.0, 1167.0)


### Reducción dimensión de tiempo

La reducción se realiza a un tiempo $T$ siguiendo los siguientes pasos para garantizar la integridad de las notas:

- Recortar tiempo muerto al inicio del MIDI
- Encontrar el punto de corte en $T$ del MIDI
- Verificar si hay una nota sonando
- Eliminar la nota sonando
- Capturar el tiempo $t_f$ en la que termina la última nota ($t_f \leq 10$)

In [23]:
def procesar_midis(midi_names, output_folder, T):
    """
    Aplica las funciones de recorte de desfase de tiempo inicial (lstrip) y ajuste a T segundos 
    a todos los archivos MIDI en una carpeta.
    
    Parámetros:
        midi_names (Series): Nombres de los MIDI a convertir.
        output_folder (str): Carpeta de destino para guardar los archivos MIDI procesados.
        T (float): Duración máxima en segundos para los archivos MIDI procesados.
    """
    # Revisar carpeta de salida
    os.makedirs(output_folder, exist_ok=True)
    # Definir carpeta de MIDIs
    input_folder = 'datos/MIDIs/midi_data/'

    
    for midi_name in midi_names:
        midi_path = os.path.join(input_folder, midi_name + '.mid')
        mid = mido.MidiFile(midi_path)
        
        # Recortar tiempo muerto
        mid = utils.midi.lstrip(mid)
        
        # Recortar duración a T segundos
        tempo = df_midi['tempo_ms'].loc[midi_name]
        mid = utils.midi.trim(mid, tempo, T)
        
        # Guardar el archivo MIDI procesado
        output_path = os.path.join(output_folder, midi_name + '.mid')
        mid.save(output_path)


In [24]:
procesar_midis(midi_names=df_midi.index,
               output_folder='datos_procesados/midis/trimmed/',
               T=10)

# Tarareos


In [1]:
import utils

In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from 'c:\\Users\\Jafet Velásquez Luna\\Documents\\Cursos\\Dip - IA Aplicada\\Proyecto Final\\hum_transcription_Project_DIA\\utils\\__init__.py'>

In [3]:
path = 'datos/Tarareos/test/'
# Duración de la ventana en milisegundos
L = 55  # por ejemplo, 500 milisegundos

# Procesar los archivos de audio y generar los espectrogramas
spectro = utils.tarareos.process_audio_files(path, L)

# Otros

In [17]:
midi = mido.MidiFile('Datos/MIDIs/midi_data/F04_0300_0001_1_D.mid')
midi

MidiFile(type=1, ticks_per_beat=1024, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=857143, time=0),
    MetaMessage('key_signature', key='C', time=0),
    MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=1024)]),
  MidiTrack([
    MetaMessage('track_name', name='é\x92¢ç\x90´, Piano', time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('pitchwheel', channel=0, pitch=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('note_on', channel=0, note=65, velocity=80, time=0),
    Message('note_off', channel=0, note=65, velocity=0, time=768),
    Message('note_on', channel=0, note=62, velocity=80, time=0),
    Message('note_off', channel=0, note=62, velocity=0, time=256),
    Message('note_on', channel=0, note=60, velocity=80, time=0),
    Message('note_o